In [1]:
from paddleocr import PaddleOCR
from PIL import Image
import io

In [2]:
model = PaddleOCR(rec=True, rec_model_dir='/home/duc-softzone/intelligent-surveillance-system/src/plate-recog/inference/rec_r45_abinet_train', use_angle_cls=True)

[2024/10/11 01:18:24] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/duc-softzone/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/duc-softzone/intelligent-surveillance-system/src/plate-recog/inference/rec_r45_abinet_train', rec_image_inverse=True, rec_image_s

In [7]:
from time import time
def rec(path):
    start_time = time()
    result = model.ocr(path, rec=True)
    try:
        text = result[0][0][0]  # Extract the text
        confidence = result[0][0][1]  # Extract confidence score
        
        end_time = time()
        print(path, text, confidence, 'Duration:', (end_time - start_time) * 1000, 'ms')
    except Exception as e:
        print(path, e)

In [4]:
import os
BASE_PATH = 'data/train/images'
dir_list = os.listdir(BASE_PATH)

In [8]:
for file in dir_list:
    path = os.path.join(BASE_PATH, file)
    rec(path)


[2024/10/11 01:19:20] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.11468100547790527
[2024/10/11 01:19:20] ppocr DEBUG: cls num  : 3, elapsed : 0.01846790313720703
[2024/10/11 01:19:20] ppocr DEBUG: rec_res num  : 3, elapsed : 0.1681053638458252
data/train/images/001074_jpg.rf.6f6406b7c8f5dd3ede891476267564eb.jpg [[125.0, 190.0], [289.0, 182.0], [292.0, 232.0], [128.0, 240.0]] ('NBD-0198', 0.9314591884613037) Duration: 312.0553493499756 ms
[2024/10/11 01:19:20] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.10318589210510254
[2024/10/11 01:19:20] ppocr DEBUG: cls num  : 3, elapsed : 0.014670133590698242
[2024/10/11 01:19:20] ppocr DEBUG: rec_res num  : 3, elapsed : 0.14493989944458008
data/train/images/001725_jpg.rf.9b6b05a541c7a6e6f1490d61e2b51a75.jpg [[174.0, 198.0], [239.0, 196.0], [240.0, 221.0], [175.0, 223.0]] ('MRY-0890', 0.9819923639297485) Duration: 270.9674835205078 ms
[2024/10/11 01:19:20] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.10193610191345215
[2024/10/11 01:19:20] ppocr DEB

KeyboardInterrupt: 

In [9]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import requests

processor = TrOCRProcessor.from_pretrained("PawanKrGunjan/license_plate_recognizer")
model = VisionEncoderDecoderModel.from_pretrained("PawanKrGunjan/license_plate_recognizer")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print('device:', device)

2024-10-11 01:19:49.708400: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-11 01:19:55.304916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


device: cuda


In [10]:
def trocr_rec(path):
    start_time = time()
    try:
        image = Image.open(path)
        pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        end_time = time()
        print(f"Image: {path}, Text: {generated_text}, Duration: {(end_time - start_time) * 1000:.2f} ms")
    except Exception as e:
        print(f"Error processing {path}: {e}")

In [11]:
for file in dir_list:
    path = os.path.join(BASE_PATH, file)
    trocr_rec(path)


Image: data/train/images/001074_jpg.rf.6f6406b7c8f5dd3ede891476267564eb.jpg, Text: OD 04W 5077, Duration: 3129.93 ms
Image: data/train/images/001725_jpg.rf.9b6b05a541c7a6e6f1490d61e2b51a75.jpg, Text: OD 09R 9477, Duration: 242.10 ms
Image: data/train/images/000883_jpg.rf.e84cb0ec653ba48ac6408013725d42c0.jpg, Text: TN 19N 7106, Duration: 235.39 ms
Image: data/train/images/000472_jpg.rf.ff91707fe85e7cc4ba68251391042ec0.jpg, Text: OD 22BZ 2476, Duration: 263.88 ms
Image: data/train/images/001604_jpg.rf.f859d1efd3834937199a6aa9b0687cb7.jpg, Text: WB 98J 5878, Duration: 233.09 ms
Image: data/train/images/001286_jpg.rf.c3fc67b0c9cdaa6a04b9e7512bc7e58f.jpg, Text: HR 17 7267, Duration: 231.52 ms
Image: data/train/images/000699_jpg.rf.af4122f2ffc9ec8d9d01c61566a951b7.jpg, Text: TN X 9579, Duration: 232.63 ms
Image: data/train/images/002875_jpg.rf.e56e9f1378124cc8c413a3efc59922dc.jpg, Text: OD 09M 8757, Duration: 235.36 ms
Image: data/train/images/001253_jpg.rf.d55b476c01d703291f23dcaec06155c3.j

KeyboardInterrupt: 

In [4]:
from ultralytics import YOLO
import cv2

In [5]:
model = YOLO("models/yolo11x.pt")

In [ ]:
results = model.train(data="/home/duc-softzone/intelligent-surveillance-system/src/plate-recog/src/datasets/data.yaml", epochs=150, batch=12, patience=20)

New https://pypi.org/project/ultralytics/8.3.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.9 🚀 Python-3.10.13 torch-2.1.2+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=detect, mode=train, model=models/yolo11x.pt, data=datasets/data.yaml, epochs=150, time=None, patience=20, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save

train: Scanning /home/duc-softzone/intelligent-surveillance-system/src/plate-recog/src/datasets/train/labels.cache... 18816 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18816/18816 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 329, len(boxes) = 25354. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /home/duc-softzone/intelligent-surveillance-system/src/plate-recog/src/datasets/valid/labels.cache... 497 images, 0 backgrounds, 0 corrupt: 100%|██████████| 497/497 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 633. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     31/150      13.6G     0.4593     0.2589     0.9287         28        640: 100%|██████████| 1568/1568 [16:12<00:00,  1.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:15<00:00,  1.37it/s]

                   all        497        633      0.984      0.981      0.993      0.912



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     32/150      13.8G     0.4616     0.2591     0.9303         25        640:  47%|████▋     | 736/1568 [07:27<08:26,  1.64it/s]

In [9]:
model.val()

Ultralytics 8.3.9 🚀 Python-3.10.13 torch-2.1.2+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /home/duc-softzone/intelligent-surveillance-system/src/plate-recog/src/datasets/valid/labels.cache... 497 images, 0 backgrounds, 0 corrupt: 100

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 633. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:11<00:00,  3.61it/s]


                   all        497        633      0.989      0.981      0.994      0.922
Speed: 0.2ms preprocess, 14.7ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train72


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a47f22158a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [5]:
from typing import List, Tuple, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torchvision.ops import batched_nms, nms


In [6]:
def det_postprocess(data: Tuple[Tensor, Tensor, Tensor, Tensor]):
    assert len(data) == 4
    iou_thres: float = 0.65
    num_dets, bboxes, scores, labels = data[0][0], data[1][0], data[2][
        0], data[3][0]
    nums = num_dets.item()
    if nums == 0:
        return bboxes.new_zeros((0, 4)), scores.new_zeros(
            (0, )), labels.new_zeros((0, ))
    # check score negative
    scores[scores < 0] = 1 + scores[scores < 0]
    # add nms
    idx = nms(bboxes, scores, iou_thres)
    bboxes, scores, labels = bboxes[idx], scores[idx], labels[idx]
    bboxes = bboxes[:nums]
    scores = scores[:nums]
    labels = labels[:nums]

    return bboxes, scores, labels

In [1]:
from plate_recognizer import PlateRecognizer
import utils

In [2]:
cfg = utils.read_config()
plate_recognizer = PlateRecognizer(cfg)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import os, cv2

test_dir = os.listdir('/home/duc-softzone/.cache/kagglehub/datasets/duydieunguyen/licenseplates/versions/1/images/val')
for file in test_dir:
    path = os.path.join('/home/duc-softzone/.cache/kagglehub/datasets/duydieunguyen/licenseplates/versions/1/images/val', file)
    image = cv2.imread(path)
    plate_recognizer.recog(image)
    # image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # results = model.predict(image)
    # for result in results:
    #     draw = image_rgb.copy()
    #     for box in result.boxes:
            
    #         draw = cv2.rectangle(draw, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
    #                       (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), 2)
    #         draw = cv2.putText(draw, f"{result.names[int(box.cls[0])]} | {box.conf[0]:.2f}",
    #                     (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
    #                     cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)
    #     plt.imshow(draw)
    #     plt.axis('off')
    #     plt.show()
            
            
            
    # break
